**Question**: Build, compile, train, and evaluate the IBM stock market model to calculate the root mean squared error loss




**Level**: Hard

**Input Format**:
Dataset


**Output Format**:
Model loss


**Sample Input**:
IBM dataset


**Sample Output**:
3.58609



**Solution**:

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error

dataset = pd.read_csv("/home/metagogy/IBM_dataset.csv",index_col="Date",parse_dates=["Date"])

training_set = dataset[:"2016"].iloc[:,1:2].values
test_set = dataset["2017":].iloc[:,1:2].values

sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

X_train = []
y_train = []

for i in range(60,2769):
    X_train.append(training_set_scaled[i-60:i,0])
    y_train.append(training_set_scaled[i,0])
X_train,y_train = np.array(X_train),np.array(y_train)

X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))

model = keras.models.Sequential()
model.add(keras.layers.GRU(units = 50, return_sequences = True, input_shape = (X_train.shape[1],1), activation = "tanh"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.GRU(units = 50, return_sequences = True, activation = "tanh"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.GRU(units = 50, return_sequences = True, activation = "tanh"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.GRU(units = 50, activation = "tanh"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(units=1))

model.compile(loss = "mean_squared_error",optimizer = keras.optimizers.SGD(lr=0.01))

model.fit(X_train,y_train,epochs = 10,batch_size=32)

dataset_total = pd.concat((dataset["High"][:"2016"],dataset["High"]["2017":]),axis=0)

inputs = dataset_total[len(dataset_total)-len(test_set) - 60:].values
inputs = inputs.reshape(-1,1)
inputs  = sc.transform(inputs)

X_test = []
for i in range(60,311):
    X_test.append(inputs[i-60:i,0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
predicted_stock_price = model.predict(X_test)

predicted_stock_price = sc.inverse_transform(predicted_stock_price)

def return_rmse(test,predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error is {}.".format(rmse))

return_rmse(test_set,predicted_stock_price)
